## Improvement
- This notebook is going to training the data separately, based on a thought that universal convergence theorem (A multiple layers neural network with at least one nonlinear layer can approximate any Borel measurable function.)
- Basic idea: Different normalization methods will result in different functions. Therefore, we need to train the data separately according to their normalization methods.

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
import os
import sys
from tqdm import tqdm

In [4]:
os.chdir(r"C:\Users\hanya\Desktop\MSBD5001 Group Project\Individual Project")

### Data Preprocessing
- First, I will do the data normalization and some feature engineering.
- Second, I would separate the data according to their penalty law.

In [5]:
training_data = pd.read_csv('./train.csv').drop('id', axis=1)
test_data = pd.read_csv('./test.csv').drop('id',axis=1)
training_data['n_jobs'].loc[training_data['n_jobs']==-1] = 16
test_data['n_jobs'].loc[test_data['n_jobs']==-1] = 16
training_label = training_data['time']
data_corr = training_data.corr()['time']

C:\Users\hanya\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
# col_drop: the columns in "col_drop" will be dropped
# col_drop = data_corr[abs(data_corr)<0.1]
# col_drop.index
col_drop = pd.Series(index=['random_state'])

In [15]:
all_data = pd.concat([training_data.drop('time',axis=1), test_data])#.drop('id',axis=1)])
all_data.drop(col_drop.index,axis=1, inplace=True)

In [17]:
### feature construction
all_data['n_samples_n_jobs'] = all_data['n_samples']/all_data['n_jobs']
all_data['max_iter_n_jobs'] = all_data['max_iter']/all_data['n_jobs']
all_data['max_iter_n_samples'] = all_data['max_iter']*all_data['n_samples']
all_data['n_samples_n_features'] = all_data['n_samples']*all_data['n_features']
all_data['n_samples_n_features_max_iter'] = all_data['n_samples']*all_data['n_features']*all_data['max_iter']

In [24]:
### Data Normalization
all_data1 = all_data.copy()
all_data = pd.concat([all_data1['penalty'],(all_data.iloc[:, 1:] - all_data.iloc[:, 1:].mean())/ all_data.iloc[:, 1:].std()], axis = 1)

In [29]:
tn_len = 440
ts_len = 100
tn_data = all_data.iloc[:tn_len, :]
ts_data = all_data.iloc[-ts_len:, :]

In [39]:
### Data Separation
# none
# 16 columns
tn_data_none = tn_data[tn_data['penalty']=='none'].drop(['penalty', 'l1_ratio'], axis=1)
ts_data_none = ts_data[ts_data['penalty']=='none'].drop(['penalty', 'l1_ratio'], axis=1)
tn_idx_none = tn_data_none.index
ts_idx_none = ts_data_none.index

# l1
# 16 columns
tn_data_l1 = tn_data[tn_data['penalty']=='l1'].drop(['penalty', 'l1_ratio'], axis=1)
ts_data_l1 = ts_data[ts_data['penalty']=='l1'].drop(['penalty', 'l1_ratio'], axis=1)
tn_idx_l1 = tn_data_l1.index
ts_idx_l1 = ts_data_l1.index

# l2
# 16 columns
tn_data_l2 = tn_data[tn_data['penalty']=='l2'].drop(['penalty', 'l1_ratio'], axis=1)
ts_data_l2 = ts_data[ts_data['penalty']=='l2'].drop(['penalty', 'l1_ratio'], axis=1)
tn_idx_l2 = tn_data_l2.index
ts_idx_l2 = ts_data_l2.index

# elasticnet
# 15 columns
tn_data_ela = tn_data[tn_data['penalty']=='elasticnet'].drop(['penalty'], axis=1)
ts_data_ela = ts_data[ts_data['penalty']=='elasticnet'].drop(['penalty'], axis=1)
tn_idx_ela = tn_data_ela.index
ts_idx_ela = ts_data_ela.index

In [111]:
### Rerference from the results before
ref = pd.read_csv('./ref.csv', engine='python')

In [105]:
### Neural Network for L1
def model_nn(X_train, y_train):#, X_train_test, y_train_test):
    x_train = X_train
    y_train = y_train
    #x_test = X_train_test
    #y_test = y_train_test

    model = Sequential()
    model.add(Dense(50, activation='tanh', input_dim=15))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    adm = Adam(lr=0.01, decay=1e-6)
    model.compile(loss='mean_squared_error',
                  optimizer=adm)

    model.fit(x_train, y_train,
              epochs=1000,
              batch_size=100)
    #score = model.evaluate(x_test, y_test, batch_size=40)
    return model#, score

In [106]:
ml1 = []
for i in tqdm(range(2)):
    ml1.append(model_nn(tn_data_l1, training_label[tn_idx_l1]))


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Epoch 1/1000
104/104 [==============================] - ETA: 0s - loss: 71.11 - 1s 9ms/step - loss: 68.4640
Epoch 2/1000
104/104 [==============================] - ETA: 0s - loss: 51.94 - 0s 96us/step - loss: 51.2657
Epoch 3/1000
104/104 [==============================] - ETA: 0s - loss: 39.60 - 0s 115us/step - loss: 38.1581
Epoch 4/1000
104/104 [==============================] - ETA: 0s - loss: 27.43 - 0s 86us/step - loss: 26.4641
Epoch 5/1000
104/104 [==============================] - ETA: 0s - loss: 19.30 - 0s 125us/step - loss: 18.6407
Epoch 6/1000
104/104 [==============================] - ETA: 0s - loss: 15.95 - 0s 192us/step - loss: 16.2445
Epoch 7/1000
104/104 [==============================] - ETA: 0s - loss: 13.20 - 0s 96us/step - loss: 13.0486
Epoch 8/1000
104/104 [==============================] - ETA: 0s - loss: 10.54 - 0s 125us/step - loss: 10.2296
Epoch 9/1000
104/104 [==============================] - ETA: 0s - loss: 7.796 - 0s 77us/step - loss: 10.4015
Epoch 10/1000
10

Epoch 76/1000
104/104 [==============================] - ETA: 0s - loss: 0.582 - 0s 201us/step - loss: 0.5688
Epoch 77/1000
104/104 [==============================] - ETA: 0s - loss: 0.473 - 0s 115us/step - loss: 0.5362
Epoch 78/1000
104/104 [==============================] - ETA: 0s - loss: 0.562 - 0s 115us/step - loss: 0.5517
Epoch 79/1000
104/104 [==============================] - ETA: 0s - loss: 0.747 - 0s 144us/step - loss: 0.7321
Epoch 80/1000
104/104 [==============================] - ETA: 0s - loss: 0.779 - 0s 125us/step - loss: 0.8139
Epoch 81/1000
104/104 [==============================] - ETA: 0s - loss: 0.642 - 0s 125us/step - loss: 0.6211
Epoch 82/1000
104/104 [==============================] - ETA: 0s - loss: 0.403 - 0s 86us/step - loss: 0.4033
Epoch 83/1000
104/104 [==============================] - ETA: 0s - loss: 0.346 - 0s 105us/step - loss: 0.3908
Epoch 84/1000
104/104 [==============================] - ETA: 0s - loss: 0.445 - 0s 105us/step - loss: 0.4547
Epoch 85/10

Epoch 224/1000
104/104 [==============================] - ETA: 0s - loss: 0.068 - 0s 125us/step - loss: 0.0783
Epoch 225/1000
104/104 [==============================] - ETA: 0s - loss: 0.040 - 0s 96us/step - loss: 0.0391
Epoch 226/1000
104/104 [==============================] - ETA: 0s - loss: 0.057 - 0s 144us/step - loss: 0.0586
Epoch 227/1000
104/104 [==============================] - ETA: 0s - loss: 0.080 - 0s 115us/step - loss: 0.0903
Epoch 228/1000
104/104 [==============================] - ETA: 0s - loss: 0.063 - 0s 105us/step - loss: 0.0642
Epoch 229/1000
104/104 [==============================] - ETA: 0s - loss: 0.099 - 0s 125us/step - loss: 0.0973
Epoch 230/1000
104/104 [==============================] - ETA: 0s - loss: 0.121 - 0s 125us/step - loss: 0.1183
Epoch 231/1000
104/104 [==============================] - ETA: 0s - loss: 0.111 - 0s 144us/step - loss: 0.1075
Epoch 232/1000
104/104 [==============================] - ETA: 0s - loss: 0.079 - 0s 96us/step - loss: 0.0776
Epo

Epoch 298/1000
104/104 [==============================] - ETA: 0s - loss: 0.038 - 0s 77us/step - loss: 0.0401
Epoch 299/1000
104/104 [==============================] - ETA: 0s - loss: 0.023 - 0s 230us/step - loss: 0.0225
Epoch 300/1000
104/104 [==============================] - ETA: 0s - loss: 0.017 - 0s 144us/step - loss: 0.0184
Epoch 301/1000
104/104 [==============================] - ETA: 0s - loss: 0.035 - 0s 134us/step - loss: 0.0343
Epoch 302/1000
104/104 [==============================] - ETA: 0s - loss: 0.038 - 0s 144us/step - loss: 0.0371
Epoch 303/1000
104/104 [==============================] - ETA: 0s - loss: 0.027 - 0s 125us/step - loss: 0.0267
Epoch 304/1000
104/104 [==============================] - ETA: 0s - loss: 0.015 - 0s 153us/step - loss: 0.0149
Epoch 305/1000
104/104 [==============================] - ETA: 0s - loss: 0.012 - 0s 153us/step - loss: 0.0163
Epoch 306/1000
104/104 [==============================] - ETA: 0s - loss: 0.014 - 0s 144us/step - loss: 0.0144
Ep

...

Epoch 1/1000
104/104 [==============================] - ETA: 0s - loss: 72.09 - 2s 23ms/step - loss: 72.1245
Epoch 2/1000
104/104 [==============================] - ETA: 0s - loss: 57.14 - 0s 134us/step - loss: 55.0283
Epoch 3/1000
104/104 [==============================] - ETA: 0s - loss: 45.84 - 0s 144us/step - loss: 44.2181
Epoch 4/1000
104/104 [==============================] - ETA: 0s - loss: 35.58 - 0s 105us/step - loss: 34.2446
Epoch 5/1000
104/104 [==============================] - ETA: 0s - loss: 27.11 - 0s 105us/step - loss: 26.2468
Epoch 6/1000
104/104 [==============================] - ETA: 0s - loss: 21.75 - 0s 105us/step - loss: 21.1625
Epoch 7/1000
104/104 [==============================] - ETA: 0s - loss: 18.31 - 0s 96us/step - loss: 17.9744
Epoch 8/1000
104/104 [==============================] - ETA: 0s - loss: 15.85 - 0s 153us/step - loss: 15.3027
Epoch 9/1000
104/104 [==============================] - ETA: 0s - loss: 13.50 - 0s 134us/step - loss: 13.0061
Epoch 10/100

104/104 [==============================] - ETA: 0s - loss: 0.186 - 0s 182us/step - loss: 0.1951
Epoch 150/1000
104/104 [==============================] - ETA: 0s - loss: 0.096 - 0s 96us/step - loss: 0.0947
Epoch 151/1000
104/104 [==============================] - ETA: 0s - loss: 0.237 - 0s 125us/step - loss: 0.2309
Epoch 152/1000
104/104 [==============================] - ETA: 0s - loss: 0.539 - 0s 105us/step - loss: 0.5467
Epoch 153/1000
104/104 [==============================] - ETA: 0s - loss: 0.334 - 0s 134us/step - loss: 0.3223
Epoch 154/1000
104/104 [==============================] - ETA: 0s - loss: 0.198 - 0s 86us/step - loss: 0.1964
Epoch 155/1000
104/104 [==============================] - ETA: 0s - loss: 0.442 - 0s 105us/step - loss: 0.4287
Epoch 156/1000
104/104 [==============================] - ETA: 0s - loss: 0.536 - 0s 115us/step - loss: 0.5263
Epoch 157/1000
104/104 [==============================] - ETA: 0s - loss: 0.264 - 0s 77us/step - loss: 0.3309
Epoch 158/1000
104/

Epoch 297/1000
104/104 [==============================] - ETA: 0s - loss: 9.813 - 0s 105us/step - loss: 9.6014
Epoch 298/1000
104/104 [==============================] - ETA: 0s - loss: 7.480 - 0s 86us/step - loss: 7.2364
Epoch 299/1000
104/104 [==============================] - ETA: 0s - loss: 3.681 - 0s 105us/step - loss: 3.5481
Epoch 300/1000
104/104 [==============================] - ETA: 0s - loss: 2.327 - 0s 125us/step - loss: 2.2624
Epoch 301/1000
104/104 [==============================] - ETA: 0s - loss: 3.487 - 0s 86us/step - loss: 3.4328
Epoch 302/1000
104/104 [==============================] - ETA: 0s - loss: 3.935 - 0s 115us/step - loss: 4.0066
Epoch 303/1000
104/104 [==============================] - ETA: 0s - loss: 2.179 - 0s 134us/step - loss: 2.1093
Epoch 304/1000
104/104 [==============================] - ETA: 0s - loss: 1.028 - 0s 115us/step - loss: 0.9956
Epoch 305/1000
104/104 [==============================] - ETA: 0s - loss: 1.676 - 0s 134us/step - loss: 1.6315
Epo

104/104 [==============================] - ETA: 0s - loss: 0.024 - 0s 105us/step - loss: 0.0297
Epoch 594/1000
104/104 [==============================] - ETA: 0s - loss: 0.067 - 0s 106us/step - loss: 0.0690
Epoch 595/1000
104/104 [==============================] - ETA: 0s - loss: 0.125 - 0s 106us/step - loss: 0.1212
Epoch 596/1000
104/104 [==============================] - ETA: 0s - loss: 0.099 - 0s 96us/step - loss: 0.0994
Epoch 597/1000
104/104 [==============================] - ETA: 0s - loss: 0.049 - 0s 96us/step - loss: 0.0480
Epoch 598/1000
104/104 [==============================] - ETA: 0s - loss: 0.065 - 0s 125us/step - loss: 0.0653
Epoch 599/1000
104/104 [==============================] - ETA: 0s - loss: 0.119 - 0s 96us/step - loss: 0.1157
Epoch 600/1000
104/104 [==============================] - ETA: 0s - loss: 0.115 - 0s 115us/step - loss: 0.1116
Epoch 601/1000
104/104 [==============================] - ETA: 0s - loss: 0.060 - 0s 96us/step - loss: 0.0618
Epoch 602/1000
104/1

Epoch 877/1000
104/104 [==============================] - ETA: 0s - loss: 0.011 - 0s 96us/step - loss: 0.0119
Epoch 878/1000
104/104 [==============================] - ETA: 0s - loss: 0.006 - 0s 115us/step - loss: 0.0071
Epoch 879/1000
104/104 [==============================] - ETA: 0s - loss: 0.006 - 0s 86us/step - loss: 0.0064
Epoch 880/1000
104/104 [==============================] - ETA: 0s - loss: 0.006 - 0s 105us/step - loss: 0.0076
Epoch 881/1000
104/104 [==============================] - ETA: 0s - loss: 0.007 - 0s 134us/step - loss: 0.0069
Epoch 882/1000
104/104 [==============================] - ETA: 0s - loss: 0.007 - 0s 96us/step - loss: 0.0073
Epoch 883/1000
104/104 [==============================] - ETA: 0s - loss: 0.011 - 0s 96us/step - loss: 0.0113
Epoch 884/1000
104/104 [==============================] - ETA: 0s - loss: 0.014 - 0s 86us/step - loss: 0.0141
Epoch 885/1000
104/104 [==============================] - ETA: 0s - loss: 0.013 - 0s 67us/step - loss: 0.0128
Epoch 8

In [107]:
rst1 = pd.DataFrame(index=ts_idx_l1, columns=range(2))
for i in range(2):
    rst1[i] = pd.DataFrame(ml1[i].predict(ts_data_l1), index=ts_idx_l1)

In [110]:
ref.to_csv('test_sep.csv')